In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re, time
from tqdm import tqdm
from google.colab import files

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"
}

def get_offer_links(page_url):
    try:
        r = requests.get(page_url, headers=headers, timeout=10)
        if r.status_code != 200:
            return []
        soup = BeautifulSoup(r.text, "html.parser")
        links = [a.get("href") for a in soup.select("a[href*='/autoturisme/anunt/']") if a.get("href")]
        return list(set([l for l in links if l.startswith("https")]))
    except:
        return []

all_links = []
for page_num in tqdm(range(1, 210)): # 100 # 165
    url = f"https://www.autovit.ro/autoturisme?page={page_num}"
    page_links = get_offer_links(url)
    all_links.extend(page_links)
    print(f"Pagina {page_num}: {len(page_links)} linkuri gasite")
    time.sleep(1)

all_links = list(set(all_links))
print(f"\n Total linkuri: {len(all_links)}")

def parse_car_page(url):
    try:
        r = requests.get(url, headers=headers, timeout=15)
        if r.status_code != 200:
            return None
        soup = BeautifulSoup(r.text, "html.parser")
        car = {"link": url}

        # Titlu, marca, model
        title = soup.find("h1")
        car["marca_model"] = title.text.strip() if title else None
        if car["marca_model"]:
            parts = car["marca_model"].split()
            car["marca"] = parts[0] if parts else None
            car["model"] = " ".join(parts[1:]) if len(parts) > 1 else None
        else:
            car["marca"] = car["model"] = None

        # Pret (valoare + moneda)
        price_value = soup.find("span", class_="offer-price__number")
        price_currency = soup.find("span", class_="offer-price__currency")
        if price_value:
            car["pret"] = price_value.text.strip() + " " + (price_currency.text.strip() if price_currency else "")
        else:
            car["pret"] = None

        # Initializare Campuri
        car.update({
            "an": None, "km": None, "combustibil": None,
            "cutie": None, "caroserie": None,
            "capacitate_cc": None, "putere_cp": None
        })

        # Anul fabricatiei
        year_tag = soup.find("p", string=re.compile(r"(19|20)\d{2}"))
        if year_tag:
            match = re.search(r"(19|20)\d{2}", year_tag.text)
            if match:
                car["an"] = match.group(0)

        # Extrage din aria-label (restul atributelor)
        for d in soup.select("div[data-testid='detail']"):
            attr = d.get("aria-label", "").strip()
            if not attr:
                continue

            if "Km" in attr:
                car["km"] = re.sub(r"[^\d]", "", attr)
            elif "Combustibil" in attr:
                car["combustibil"] = attr.replace("Combustibil", "").strip()
            elif "Cutie" in attr:
                car["cutie"] = attr.replace("Cutie de viteze", "").strip()
            elif "Caroserie" in attr:
                car["caroserie"] = attr.replace("Tip Caroserie", "").strip()
            elif "cm3" in attr or "cilindric" in attr:
                car["capacitate_cc"] = re.sub(r"[^\d]", "", attr)
            elif "Putere" in attr:
                car["putere_cp"] = re.sub(r"[^\d]", "", attr)

        return car

    except Exception as e:
        print(f"[Eroare la {url}] — {type(e).__name__}: {e}")
        return None


cars_data = []
for i, url in tqdm(enumerate(all_links, 1), total=len(all_links)):
    car = parse_car_page(url)
    if car:
        cars_data.append(car)
    time.sleep(0.8)

df = pd.DataFrame(cars_data)
df.to_csv("/content/autovit_final_dataset.csv", index=False)

print(f"\n Am colectat {len(df)} masini.")
df.head()

files.download("/content/autovit_final_dataset.csv")

  0%|          | 0/209 [00:00<?, ?it/s]

Pagina 1: 32 linkuri gasite


  0%|          | 1/209 [00:03<10:25,  3.01s/it]

Pagina 2: 32 linkuri gasite


  1%|          | 2/209 [00:07<12:46,  3.70s/it]

...


 Total linkuri: 6041


100%|██████████| 6041/6041 [4:15:32<00:00,  2.54s/it]


 Am colectat 6034 masini.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>